In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nyc_df = pd.read_csv('.\dataset_tsmc2014\dataset_TSMC2014_NYC.txt', delimiter = "\t", names=["user_id", "venue_id", "venue_category_id", "venue_category_name", "latitude", "longitude", "timezone_offset", "UTC_time"])
nyc_df['cleaned_UTC_time'] = pd.to_datetime(nyc_df["UTC_time"])
cleaned_nyc_df = nyc_df.drop_duplicates(subset=['user_id', 'venue_id', 'UTC_time'], keep='first')
cleaned_nyc_df

,user_id,venue_id,venue_category_id,venue_category_name,latitude,longitude,timezone_offset,UTC_time,cleaned_UTC_time
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012,2012-04-03 18:00:09+00:00
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012,2012-04-03 18:00:25+00:00
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012,2012-04-03 18:02:24+00:00
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012,2012-04-03 18:02:41+00:00
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012,2012-04-03 18:03:00+00:00
...,...,...,...,...,...,...,...,...,...
227423,688,3fd66200f964a52000e71ee3,4bf58dd8d48988d1e7931735,Music Venue,40.733596,-74.003139,-300,Sat Feb 16 02:29:11 +0000 2013,2013-02-16 02:29:11+00:00
227424,560,4bca32ff0687ef3be789dbcc,4bf58dd8d48988d16c941735,Burger Joint,40.745719,-73.993720,-300,Sat Feb 16 02:31:35 +0000 2013,2013-02-16 02:31:35+00:00
227425,945,50a77716e4b0b5a9492f6f56,4bf58dd8d48988d103941735,Home (private),40.854364,-73.883070,-300,Sat Feb 16 02:33:16 +0000 2013,2013-02-16 02:33:16+00:00
227426,671,4514efe0f964a520e7391fe3,4bf58dd8d48988d11d941735,Bar,40.735981,-74.029309,-300,Sat Feb 16 02:34:31 +0000 2013,2013-02-16 02:34:31+00:00


### Further cleaning of general categories information

In [3]:
cleaned_nyc_df.loc[cleaned_nyc_df.venue_category_name == 'Caf�', 'venue_category_name'] = 'Cafe'
cleaned_nyc_df.loc[cleaned_nyc_df.venue_category_name == 'Home (private)', 'venue_category_name'] = 'Home'

removing_list = ['Nightlife Spot', 'Taxi', 'Other Nightlife', 'Moving Target', 'General Travel', 'Professional & Other Places', 'Harbor / Marina', 'Parking',
                 'Subway', 'Road', 'River', 'LightRail', 'City', 'Ferry', 'Bridge', 'Neighborhood']

for i in removing_list:
    cleaned_nyc_df = cleaned_nyc_df[cleaned_nyc_df.venue_category_name != i]

D:\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [4]:
cleaned_nyc_df_more_categories = cleaned_nyc_df.copy()

First construct the original dataset (10 categories)

In [6]:
eateries = ['Afghan Restaurant', 'Gluten-free Restaurant', 'Portuguese Restaurant', 'Fish & Chips Shop', 'Swiss Restaurant',
            'Moroccan Restaurant', 'Ethiopian Restaurant', 'Arepa Restaurant', 'Turkish Restaurant', 'Scandinavian Restaurant',
            'Argentinian Restaurant', 'Peruvian Restaurant', 'Mac & Cheese Joint', 'Filipino Restaurant', 'African Restaurant',
            'Malaysian Restaurant', 'Dumpling Restaurant', 'Cajun / Creole Restaurant', 'Dim Sum Restaurant',
            'Molecular Gastronomy Restaurant', 'Australian Restaurant', 'Eastern European Restaurant', 'Brazilian Restaurant',
            'South American Restaurant', 'Falafel Restaurant', 'Greek Restaurant', 'Wings Joint', 'Southern / Soul Food Restaurant',
            'Soup Place', 'Food', 'Mediterranean Restaurant', 'Vietnamese Restaurant', 'German Restaurant', 'Tapas Restaurant',
            'Middle Eastern Restaurant', 'Salad Place', 'Taco Place', 'Tea Room', 'Spanish Restaurant', 'Cupcake Shop',
            'Hot Dog Joint', 'Burrito Place', 'Ramen/Noodle House', 'Gastropub', 'Vegetarian / Vegan Restaurant',
            'Cuban Restaurant', 'Steakhouse', 'Indian Restaurant', 'Latin American Restaurant', 'Fried Chicken Joint',
            'Korean Restaurant', 'Caribbean Restaurant', 'Japanese Restaurant', 'Dessert Shop', 'Breakfast Spot', 'Beer Garden',
            'Thai Restaurant', 'Restaurant', 'Asian Restaurant', 'Seafood Restaurant', 'BBQ Joint', 'French Restaurant',
            'Food Truck', 'Ice Cream Shop', 'Bagel Shop', 'Sushi Restaurant', 'Donut Shop', 'Diner', 'Chinese Restaurant',
            'Cafe', 'Fast Food Restaurant', 'Burger Joint', 'Sandwich Place', 'Italian Restaurant', 'Mexican Restaurant',
            'Pizza Place', 'American Restaurant', 'Deli / Bodega', 'Food & Drink Shop', 'Coffee Shop', 'Bar', 'Snack Place', 'Bakery']

exhibitions = ['Public Art', 'Castle', 'Historic Site', 'Planetarium', 'Aquarium', 'Science Museum', 'History Museum', 'Museum',
               'Art Museum', 'Zoo', 'Fair', 'Convention Center', 'Event Space']

religious_places = ['Mosque', 'Shrine', 'Temple', 'Synagogue', 'Church', 'Spiritual Center']

shops = ['Motorcycle Shop', 'Newsstand','Board Shop', 'Antique Shop', 'Car Dealership', 'Camera Store', 'Shop & Service', 'Video Store', 
         'Hobby Shop', 'Bridal Shop', 'Music Store','Flower Shop', 'Mobile Phone Shop',
         'Record Shop', 'Market', 'Jewelry Store', 'Video Game Store', 'Gift Shop', 'Bike Shop',
         'Thrift / Vintage Store', 'Candy Store', 'Smoke Shop', 'Toy / Game Store', 'Flea Market', 'Arts & Crafts Store',
         'Paper / Office Supplies Store', 'Pet Store', 'Sporting Goods Shop', 'Hardware Store', 'Cosmetics Shop', 'Furniture / Home Store',
         'Automotive Shop', 'Convenience Store', 'Bookstore', 'Miscellaneous Shop',
         'Electronics Store', 'Department Store', 'Drugstore / Pharmacy', 'Clothing Store', 'Garden Center']

services = ['Photography lab', 'Design studio', 'Cemetery', 'Library', 'Storage Facility', 'Financial or Legal Service', 'Recycling Facility',
            'Bank', 'Hotel', 'Laundry Service', 'Post Office', 'Housing Development', 'Rest Area', 'Student Center', 'Pet Service', 
            'Internet Cafe', 'Bike Rental / Bike Share', 'Gaming Cafe', 'Car Wash', 'Rental Car Location', 'Animal Shelter', 
            'Funeral Home', 'Nail Salon', 'Tanning Salon', 'Tattoo Parlor', 'Spa / Massage', 'Mall', 'Gas Station / Garage', 
            'Salon / Barbershop', 'Plaza', 'Medical Center']


education = ['Music School', 'College Theater', 'Medical School', 'Middle School',
             'College & University', 'Trade School', 'College Stadium', 'Law School', 'Nursery School', 'Elementary School',
            'Community College', 'General College & University', 'High School', 'University', 'School',
             'College Academic Building']

arts = ['Concert Hall', 'Art Gallery', 'Performing Arts Venue', 'Music Venue']

outdoor = ['Other Great Outdoors', 'Scenic Lookout', 'Garden', 'Park', 'Beach', 'Outdoors & Recreation', 'Campground', 'Sculpture Garden']

entertainment = ['Arts & Entertainment', 'Movie Theater', 'General Entertainment', 'Stadium', 'Arcade',
                 'Bowling Alley', 'Pool', 'Comedy Club', 'Casino', 'Pool Hall', 'Racetrack', 'Ski Area', 'Theater']

fitness = ['Athletic & Sport', 'Gym / Fitness Center']

In [7]:
general_categories_dict = {
    'Eateries': eateries,
    'Exhibitions': exhibitions,
    'Religious Places': religious_places,
    'Shops': shops,
    'Services': services,
    'Education': education,
    'Arts': arts,
    'Outdoor': outdoor,
    'Entertainment': entertainment,
    'Fitness': fitness
}

def add_general_category_information(row):
    for i in general_categories_dict:
        if row['venue_category_name'] in general_categories_dict[i]:
            return i
    return ''

cleaned_nyc_df['general_venue_category_name'] = cleaned_nyc_df.apply(add_general_category_information, axis=1)

In [8]:
cleaned_nyc_df.head()

,user_id,venue_id,venue_category_id,venue_category_name,latitude,longitude,timezone_offset,UTC_time,cleaned_UTC_time,general_venue_category_name
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012,2012-04-03 18:00:09+00:00,Shops
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home,40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012,2012-04-03 18:02:24+00:00,
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012,2012-04-03 18:02:41+00:00,Services
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012,2012-04-03 18:03:00+00:00,Eateries
5,484,4b5b981bf964a520900929e3,4bf58dd8d48988d118951735,Food & Drink Shop,40.690427,-73.954687,-240,Tue Apr 03 18:04:00 +0000 2012,2012-04-03 18:04:00+00:00,Eateries


In [9]:
cleaned_nyc_df = cleaned_nyc_df[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'general_venue_category_name', 'latitude',
                                 'longitude', 'UTC_time']]
cleaned_nyc_df.to_csv('cleaned_NYC_records_original_updated.csv', index=False)

Then the one with more categories (13 categories)

In [10]:
eateries = ['Afghan Restaurant', 'Gluten-free Restaurant', 'Portuguese Restaurant', 'Fish & Chips Shop', 'Swiss Restaurant',
            'Moroccan Restaurant', 'Ethiopian Restaurant', 'Arepa Restaurant', 'Turkish Restaurant', 'Scandinavian Restaurant',
            'Argentinian Restaurant', 'Peruvian Restaurant', 'Mac & Cheese Joint', 'Filipino Restaurant', 'African Restaurant',
            'Malaysian Restaurant', 'Dumpling Restaurant', 'Cajun / Creole Restaurant', 'Dim Sum Restaurant',
            'Molecular Gastronomy Restaurant', 'Australian Restaurant', 'Eastern European Restaurant', 'Brazilian Restaurant',
            'South American Restaurant', 'Falafel Restaurant', 'Greek Restaurant', 'Wings Joint', 'Southern / Soul Food Restaurant',
            'Soup Place', 'Food', 'Mediterranean Restaurant', 'Vietnamese Restaurant', 'German Restaurant', 'Tapas Restaurant',
            'Middle Eastern Restaurant', 'Salad Place', 'Taco Place', 'Spanish Restaurant',
            'Hot Dog Joint', 'Burrito Place', 'Ramen/Noodle House', 'Gastropub', 'Vegetarian / Vegan Restaurant',
            'Cuban Restaurant', 'Steakhouse', 'Indian Restaurant', 'Latin American Restaurant', 'Fried Chicken Joint',
            'Korean Restaurant', 'Caribbean Restaurant', 'Japanese Restaurant',
            'Thai Restaurant', 'Restaurant', 'Asian Restaurant', 'Seafood Restaurant', 'BBQ Joint', 'French Restaurant',
            'Food Truck', 'Sushi Restaurant', 'Diner', 'Chinese Restaurant', 'Fast Food Restaurant', 'Burger Joint', 
            'Sandwich Place', 'Italian Restaurant', 'Mexican Restaurant',
            'Pizza Place', 'American Restaurant', 'Deli / Bodega', 'Food & Drink Shop', 'Snack Place']

cafe_dessert = ['Tea Room', 'Cupcake Shop', 'Dessert Shop', 'Breakfast Spot', 'Ice Cream Shop', 'Bagel Shop', 'Donut Shop',
                'Cafe', 'Coffee Shop', 'Bakery']

drinking_place = ['Bar', 'Beer Garden']

indoor_exhibitions = ['Planetarium', 'Aquarium', 'Science Museum', 'History Museum', 'Museum',
                       'Art Museum', 'Convention Center', 'Event Space']

outdoor_exhibitions = ['Public Art', 'Castle', 'Historic Site', 'Zoo', 'Fair']

religious_places = ['Mosque', 'Shrine', 'Temple', 'Synagogue', 'Church', 'Spiritual Center']

shops = ['Motorcycle Shop', 'Newsstand',
         'Board Shop', 'Antique Shop', 'Car Dealership', 'Camera Store', 'Shop & Service', 'Video Store', 'Hobby Shop', 'Bridal Shop', 
         'Music Store', 'Flower Shop', 'Mobile Phone Shop',
         'Record Shop', 'Market', 'Jewelry Store', 'Video Game Store', 'Gift Shop', 'Bike Shop',
         'Thrift / Vintage Store', 'Candy Store', 'Smoke Shop', 'Toy / Game Store', 'Flea Market', 'Arts & Crafts Store',
         'Paper / Office Supplies Store', 'Pet Store', 'Sporting Goods Shop', 'Hardware Store', 'Cosmetics Shop', 'Furniture / Home Store',
         'Automotive Shop', 'Convenience Store', 'Bookstore', 'Miscellaneous Shop',
         'Electronics Store', 'Department Store', 'Drugstore / Pharmacy', 'Clothing Store', 'Garden Center']

services = ['Photography lab', 'Design studio', 'Cemetery', 'Library', 'Storage Facility', 'Financial or Legal Service', 'Recycling Facility',
            'Bank', 'Hotel', 'Laundry Service', 'Post Office', 'Housing Development', 'Rest Area', 'Student Center', 'Pet Service', 
            'Internet Cafe', 'Bike Rental / Bike Share', 'Gaming Cafe', 'Car Wash', 'Rental Car Location', 'Animal Shelter', 'Funeral Home', 
            'Nail Salon', 'Tanning Salon', 'Tattoo Parlor', 'Spa / Massage', 'Mall', 'Gas Station / Garage', 'Salon / Barbershop', 'Plaza', 
            'Medical Center']


education = ['Music School', 'College Theater', 'Medical School', 'Middle School',
             'College & University', 'Trade School', 'College Stadium', 'Law School', 'Nursery School', 'Elementary School', 'Community College', 
             'General College & University', 'High School', 'University', 'School',
             'College Academic Building']

arts = ['Concert Hall', 'Art Gallery', 'Performing Arts Venue', 'Music Venue']

outdoor = ['Other Great Outdoors', 'Scenic Lookout', 'Garden', 'Park', 'Beach', 'Outdoors & Recreation', 'Campground', 'Sculpture Garden']

entertainment = ['Arts & Entertainment', 'Movie Theater', 'General Entertainment', 'Stadium', 'Arcade',
                 'Bowling Alley', 'Pool', 'Comedy Club', 'Casino', 'Pool Hall', 'Racetrack', 'Ski Area', 'Theater']

fitness = ['Athletic & Sport', 'Gym / Fitness Center']

In [11]:
general_categories_dict = {
    'Eateries': eateries,
    'Cafe/Desserts': cafe_dessert,
    'Drinking place': drinking_place,
    'Indoor Exhibitions': indoor_exhibitions,
    'Outdoor Exhibitions': outdoor_exhibitions,
    'Religious Places': religious_places,
    'Shops': shops,
    'Services': services,
    'Education': education,
    'Arts': arts,
    'Outdoor': outdoor,
    'Entertainment': entertainment,
    'Fitness': fitness
}

def add_general_category_information(row):
    for i in general_categories_dict:
        if row['venue_category_name'] in general_categories_dict[i]:
            return i
    return ''

cleaned_nyc_df_more_categories['general_venue_category_name'] = cleaned_nyc_df_more_categories.apply(add_general_category_information, axis=1)

In [12]:
cleaned_nyc_df_more_categories.head()

,user_id,venue_id,venue_category_id,venue_category_name,latitude,longitude,timezone_offset,UTC_time,cleaned_UTC_time,general_venue_category_name
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012,2012-04-03 18:00:09+00:00,Shops
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home,40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012,2012-04-03 18:02:24+00:00,
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012,2012-04-03 18:02:41+00:00,Services
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012,2012-04-03 18:03:00+00:00,Eateries
5,484,4b5b981bf964a520900929e3,4bf58dd8d48988d118951735,Food & Drink Shop,40.690427,-73.954687,-240,Tue Apr 03 18:04:00 +0000 2012,2012-04-03 18:04:00+00:00,Eateries


In [13]:
cleaned_nyc_df_more_categories = cleaned_nyc_df_more_categories[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'general_venue_category_name', 'latitude',
                                 'longitude', 'UTC_time']]
cleaned_nyc_df_more_categories.to_csv('cleaned_NYC_records_13_categories_updated.csv', index=False)

### Add in the duration information

For the dataset with original set of categories (10 categories)

In [14]:
def calculating_duration(row, dataframe):
    next_records_df = dataframe[(dataframe.user_id == row.user_id) & (dataframe.cleaned_UTC_time > row.cleaned_UTC_time) 
                                 & (dataframe.venue_id != row.venue_id)]

    if next_records_df.size > 1:
        next_record = next_records_df.iloc[[0]]
        c = next_record.cleaned_UTC_time - row.cleaned_UTC_time
        
        return c[c.index[0]].seconds / 3600
    
    return -1

In [15]:
new_nyc_df = pd.read_csv('cleaned_NYC_records_original_updated.csv')
new_nyc_df['cleaned_UTC_time'] = pd.to_datetime(new_nyc_df["UTC_time"])
new_nyc_df['duration'] = new_nyc_df.apply(calculating_duration, axis=1, dataframe=new_nyc_df)
new_nyc_df.head()

,user_id,venue_id,venue_category_id,venue_category_name,general_venue_category_name,latitude,longitude,UTC_time,cleaned_UTC_time,duration
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,Shops,40.719810,-74.002581,Tue Apr 03 18:00:09 +0000 2012,2012-04-03 18:00:09+00:00,5.606944
1,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home,NaN,40.716162,-73.883070,Tue Apr 03 18:02:24 +0000 2012,2012-04-03 18:02:24+00:00,1.542222
2,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,Services,40.745164,-73.982519,Tue Apr 03 18:02:41 +0000 2012,2012-04-03 18:02:41+00:00,8.194444
3,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,Eateries,40.740104,-73.989658,Tue Apr 03 18:03:00 +0000 2012,2012-04-03 18:03:00+00:00,0.070833
4,484,4b5b981bf964a520900929e3,4bf58dd8d48988d118951735,Food & Drink Shop,Eateries,40.690427,-73.954687,Tue Apr 03 18:04:00 +0000 2012,2012-04-03 18:04:00+00:00,5.985833


In [16]:
new_nyc_write_df = new_nyc_df[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'general_venue_category_name', 'latitude',
                               'longitude', 'UTC_time', 'duration']]
new_nyc_write_df.to_csv('cleaned_NYC_records_original_updated_duration.csv', index=False)

For the dataset with new set of categories (13 categories)

In [17]:
new_nyc_df_more_categories = pd.read_csv('cleaned_NYC_records_13_categories_updated.csv')
new_nyc_df_more_categories['cleaned_UTC_time'] = pd.to_datetime(new_nyc_df_more_categories["UTC_time"])
new_nyc_df_more_categories['duration'] = new_nyc_df_more_categories.apply(calculating_duration, axis=1, dataframe=new_nyc_df_more_categories)
new_nyc_df_more_categories.head()

,user_id,venue_id,venue_category_id,venue_category_name,general_venue_category_name,latitude,longitude,UTC_time,cleaned_UTC_time,duration
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,Shops,40.719810,-74.002581,Tue Apr 03 18:00:09 +0000 2012,2012-04-03 18:00:09+00:00,5.606944
1,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home,NaN,40.716162,-73.883070,Tue Apr 03 18:02:24 +0000 2012,2012-04-03 18:02:24+00:00,1.542222
2,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,Services,40.745164,-73.982519,Tue Apr 03 18:02:41 +0000 2012,2012-04-03 18:02:41+00:00,8.194444
3,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,Eateries,40.740104,-73.989658,Tue Apr 03 18:03:00 +0000 2012,2012-04-03 18:03:00+00:00,0.070833
4,484,4b5b981bf964a520900929e3,4bf58dd8d48988d118951735,Food & Drink Shop,Eateries,40.690427,-73.954687,Tue Apr 03 18:04:00 +0000 2012,2012-04-03 18:04:00+00:00,5.985833


In [18]:
new_nyc_df_more_categories_write_df = new_nyc_df_more_categories[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'general_venue_category_name', 'latitude',
                               'longitude', 'UTC_time', 'duration']]
new_nyc_df_more_categories_write_df.to_csv('cleaned_NYC_records_13_categories_updated_duration.csv', index=False)

### Add in Others Category

In [19]:
new_nyc_df = pd.read_csv('cleaned_NYC_records_original_updated_duration.csv')
new_nyc_df['cleaned_UTC_time'] = pd.to_datetime(new_nyc_df["UTC_time"])
new_nyc_df.loc[pd.isna(new_nyc_df.general_venue_category_name), 'general_venue_category_name'] = 'Others'

new_nyc_write_df = new_nyc_df[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'general_venue_category_name', 'latitude',
                               'longitude', 'UTC_time', 'duration']]
new_nyc_write_df.to_csv('cleaned_NYC_records_original_updated_duration.csv', index=False)

In [20]:
new_nyc_df_more_categories = pd.read_csv('cleaned_NYC_records_13_categories_updated_duration.csv')
new_nyc_df_more_categories['cleaned_UTC_time'] = pd.to_datetime(new_nyc_df_more_categories["UTC_time"])
new_nyc_df_more_categories.loc[pd.isna(new_nyc_df_more_categories.general_venue_category_name), 'general_venue_category_name'] = 'Others'

new_nyc_write_df = new_nyc_df_more_categories[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'general_venue_category_name', 'latitude',
                               'longitude', 'UTC_time', 'duration']]
new_nyc_write_df.to_csv('cleaned_NYC_records_13_categories_updated_duration.csv', index=False)

### Making a clean duration file without general category information

In [3]:
original_duration_df = pd.read_csv('cleaned_NYC_records_duration.csv')
original_duration_df['cleaned_UTC_time'] = pd.to_datetime(original_duration_df["UTC_time"])

cleaned_duration_df = original_duration_df[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'latitude',
                               'longitude', 'UTC_time', 'duration']]
cleaned_duration_df.to_csv('cleaned_NYC_records_duration_without_general_categories.csv')

### Making file for 7 general categories

In [4]:
cleaned_duration_df = pd.read_csv('cleaned_NYC_records_duration_without_general_categories.csv')
cleaned_duration_df['cleaned_UTC_time'] = pd.to_datetime(cleaned_duration_df["UTC_time"])

In [5]:
eateries = ['Afghan Restaurant', 'Gluten-free Restaurant', 'Portuguese Restaurant', 'Fish & Chips Shop', 'Swiss Restaurant',
            'Moroccan Restaurant', 'Ethiopian Restaurant', 'Arepa Restaurant', 'Turkish Restaurant', 'Scandinavian Restaurant',
            'Argentinian Restaurant', 'Peruvian Restaurant', 'Mac & Cheese Joint', 'Filipino Restaurant', 'African Restaurant',
            'Malaysian Restaurant', 'Dumpling Restaurant', 'Cajun / Creole Restaurant', 'Dim Sum Restaurant',
            'Molecular Gastronomy Restaurant', 'Australian Restaurant', 'Eastern European Restaurant', 'Brazilian Restaurant',
            'South American Restaurant', 'Falafel Restaurant', 'Greek Restaurant', 'Wings Joint', 'Southern / Soul Food Restaurant',
            'Soup Place', 'Food', 'Mediterranean Restaurant', 'Vietnamese Restaurant', 'German Restaurant', 'Tapas Restaurant',
            'Middle Eastern Restaurant', 'Salad Place', 'Taco Place', 'Tea Room', 'Spanish Restaurant', 'Cupcake Shop',
            'Hot Dog Joint', 'Burrito Place', 'Ramen/Noodle House', 'Gastropub', 'Vegetarian / Vegan Restaurant',
            'Cuban Restaurant', 'Steakhouse', 'Indian Restaurant', 'Latin American Restaurant', 'Fried Chicken Joint',
            'Korean Restaurant', 'Caribbean Restaurant', 'Japanese Restaurant', 'Dessert Shop', 'Breakfast Spot', 'Beer Garden',
            'Thai Restaurant', 'Restaurant', 'Asian Restaurant', 'Seafood Restaurant', 'BBQ Joint', 'French Restaurant',
            'Food Truck', 'Ice Cream Shop', 'Bagel Shop', 'Sushi Restaurant', 'Donut Shop', 'Diner', 'Chinese Restaurant',
            'Cafe', 'Fast Food Restaurant', 'Burger Joint', 'Sandwich Place', 'Italian Restaurant', 'Mexican Restaurant',
            'Pizza Place', 'American Restaurant', 'Deli / Bodega', 'Food & Drink Shop', 'Coffee Shop', 'Bar', 'Snack Place', 'Bakery']

shops = ['Motorcycle Shop', 'Newsstand','Board Shop', 'Antique Shop', 'Car Dealership', 'Camera Store', 'Shop & Service', 'Video Store', 
         'Hobby Shop', 'Bridal Shop', 'Music Store','Flower Shop', 'Mobile Phone Shop',
         'Record Shop', 'Market', 'Jewelry Store', 'Video Game Store', 'Gift Shop', 'Bike Shop',
         'Thrift / Vintage Store', 'Candy Store', 'Smoke Shop', 'Toy / Game Store', 'Flea Market', 'Arts & Crafts Store',
         'Paper / Office Supplies Store', 'Pet Store', 'Sporting Goods Shop', 'Hardware Store', 'Cosmetics Shop', 'Furniture / Home Store',
         'Automotive Shop', 'Convenience Store', 'Bookstore', 'Miscellaneous Shop',
         'Electronics Store', 'Department Store', 'Drugstore / Pharmacy', 'Clothing Store', 'Garden Center', 'Photography lab', 'Design studio', 
         'Cemetery', 'Library', 'Storage Facility', 'Financial or Legal Service', 'Recycling Facility',
         'Bank', 'Hotel', 'Laundry Service', 'Post Office', 'Housing Development', 'Rest Area', 'Student Center', 'Pet Service', 
         'Internet Cafe', 'Bike Rental / Bike Share', 'Gaming Cafe', 'Car Wash', 'Rental Car Location', 'Animal Shelter', 
         'Funeral Home', 'Nail Salon', 'Tanning Salon', 'Tattoo Parlor', 'Spa / Massage', 'Mall', 'Gas Station / Garage', 
         'Salon / Barbershop', 'Plaza', 'Medical Center']

education = ['Music School', 'College Theater', 'Medical School', 'Middle School',
             'College & University', 'Trade School', 'College Stadium', 'Law School', 'Nursery School', 'Elementary School',
            'Community College', 'General College & University', 'High School', 'University', 'School',
             'College Academic Building', 'Mosque', 'Shrine', 'Temple', 'Synagogue', 'Church', 'Spiritual Center']


outdoor = ['Other Great Outdoors', 'Scenic Lookout', 'Garden', 'Park', 'Beach', 'Outdoors & Recreation', 'Campground', 'Sculpture Garden']

entertainment = ['Arts & Entertainment', 'Movie Theater', 'General Entertainment', 'Stadium', 'Arcade',
                 'Bowling Alley', 'Pool', 'Comedy Club', 'Casino', 'Pool Hall', 'Racetrack', 'Ski Area', 'Theater',
                 'Concert Hall', 'Art Gallery', 'Performing Arts Venue', 'Music Venue', 'Public Art', 'Castle', 
                 'Historic Site', 'Planetarium', 'Aquarium', 'Science Museum', 'History Museum', 'Museum',
                 'Art Museum', 'Zoo', 'Fair', 'Convention Center', 'Event Space']

fitness = ['Athletic & Sport', 'Gym / Fitness Center']

In [6]:
general_categories_dict = {
    'Eateries': eateries,
    'Shops': shops,
    'Education': education,
    'Outdoor': outdoor,
    'Entertainment': entertainment,
    'Fitness': fitness
}

def add_general_category_information(row):
    for i in general_categories_dict:
        if row['venue_category_name'] in general_categories_dict[i]:
            return i
    return ''

cleaned_duration_df['general_venue_category_name'] = cleaned_duration_df.apply(add_general_category_information, axis=1)
cleaned_duration_df.head()

,Unnamed: 0,user_id,venue_id,venue_category_id,venue_category_name,latitude,longitude,UTC_time,duration,cleaned_UTC_time,general_venue_category_name
0,0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,Tue Apr 03 18:00:09 +0000 2012,5.606944,2012-04-03 18:00:09+00:00,Shops
1,1,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home,40.716162,-73.883070,Tue Apr 03 18:02:24 +0000 2012,1.542222,2012-04-03 18:02:24+00:00,
2,2,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,Tue Apr 03 18:02:41 +0000 2012,8.194444,2012-04-03 18:02:41+00:00,Shops
3,3,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,Tue Apr 03 18:03:00 +0000 2012,0.070833,2012-04-03 18:03:00+00:00,Eateries
4,4,484,4b5b981bf964a520900929e3,4bf58dd8d48988d118951735,Food & Drink Shop,40.690427,-73.954687,Tue Apr 03 18:04:00 +0000 2012,5.985833,2012-04-03 18:04:00+00:00,Eateries


In [14]:
cleaned_duration_7_categories_df = cleaned_duration_df.copy()
cleaned_duration_7_categories_df.loc[cleaned_duration_7_categories_df.general_venue_category_name == '', 'general_venue_category_name'] = 'Others'

new_nyc_write_df = cleaned_duration_7_categories_df[['user_id', 'venue_id', 'venue_category_id', 'venue_category_name', 'general_venue_category_name', 'latitude',
                               'longitude', 'UTC_time', 'duration']]
new_nyc_write_df.to_csv('cleaned_NYC_records_7_categories_updated_duration.csv', index=False)